---
layout: post
title:  "Ch02.Prompt"
date:   2015-06-26 02:00:00 +0700
categories: [LLM]
---

<script type="text/x-mathjax-config">
MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
</script>
<script type="text/javascript" src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-MML-AM_CHTML">
</script>

### 참조 사이트.
해당 Post는 <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a>에 나와있는 예제와 흐름을 파악하는 용도 입니다.  

### Prompt

LLM의 Input으로 들어가게 되는 Prompt에 대해 자세히 알아본다.

Prompt란 아래와 같이 정의하고 있다.

>프롬프트 단계는 검색기에서 검색된 문서들을 바탕으로 **언어 모델이 사용할 질문이나 명령을 생성하는 과정**입니다. 이 단계는 **검색된 정보를 바탕**으로 최종 사용자의 질문에 가장 잘 대응할 수 있는 응답을 생성하기 위해 필수적인 단계입니다.

Prompt는 아래와 같은 필요성을 가지고 있다.
- 문맥(Context) 설정: LLM에 추가적인 정보를 전달함으로서 할루시네이션을 줄이는 역할로 사용 가능하다.
- 정보 통합: 서로 다른 여러 정보를 하나로 합쳐 정보를 전달 할 수 있다.
- 응답 품질 향상: RAG에서 많이 사용되는 방법으로서, 도움이 되는 Context를 선택하여 전달 할 수 있다.

기본적인 RAG에서 많이 사용되는 구조는 아래와 같다.
- Instruction: 지시사항으로서, LLM에 어떤 역할로서 답변을 얻어낼지 지정한다.
- Question: 사용자의 질문을 전달한다.
- Context: RAG로서 검색된 정보를 전달한다. 답변생성에 있어서 할루시네이션을 줄이는 역할을 한다.

기본적인 RAG의 Prompt를 확인하면 아래와 같다.

```python
당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요. 
만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. 
한국어로 대답하세요.

#Question: 
{이곳에 사용자가 입력한 질문이 삽입됩니다}

#Context: 
{이곳에 검색된 정보가 삽입됩니다}
```

### Setting

#### Import Library

In [2]:
import yaml
from typing import List
from pathlib import Path
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain.embeddings.base import Embeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, load_prompt
from langchain_core.output_parsers import StrOutputParser

#### Model 선언

In [3]:
model = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model="Qwen/Qwen3-4B"
)

#### 사용할 Prompt 저장.

사용할 Prompt를 미리 지정하여 저장한다.

Appendix: OpenAI에서 제공하는 default prompt가 있으나, 실제 해당 값과 동일하게 저장하고 불러와서 확인하다.

fruit_color.yaml

In [4]:
prompt_data = {
    "_type": "prompt",
    "input_variables": ["fruit"],
    "template": "What color is a {fruit}?"
}

Path("prompts").mkdir(exist_ok=True)
with open("./data/fruit_color.yaml", "w", encoding="utf-8") as f:
    yaml.dump(prompt_data, f, allow_unicode=True)

load_prompt로 불러와서 확인

In [5]:
prompt = load_prompt("./data/fruit_color.yaml")
print(prompt)

input_variables=['fruit'] input_types={} partial_variables={} template='What color is a {fruit}?'


실제 load_prompt를 확인하게 되면, 아래와 같이 type이 **langchain_core.prompts.prompt.PromptTemplate**인 것을 확인할 수 있다.  
위와 같이 바뀌는 이유는 LangChain인 내부에서 _type을 확인하고 자동으로 Mapping하여 가져오기 때문이다.

```python
if _type == "prompt":
    return PromptTemplate(...)
```

In [6]:
type(prompt)

langchain_core.prompts.prompt.PromptTemplate

### Prompt Template

#### 방법 1. from_template() 메소드를 사용하여 PrompteTemplate 객체를 생성

In [ ]:
# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
print(type(prompt))
print(prompt)

# chain 생성
chain = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content.split('</think>\n\n')[1]

<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['country'] input_types={} partial_variables={} template='{country}의 수도는 어디인가요?'


"<think>\nOkay, the user is asking about the capital of South Korea. I need to make sure I provide the correct answer. Let me recall, I think it's Seoul. But wait, I should double-check. South Korea's capital is indeed Seoul. However, I should also mention that the administrative capital is Seoul, and there's a new administrative capital called Gwangju, but I think that's not yet completed. Wait, no, actually, I think the new capital is called Incheon, but that's not right. Wait, no, the new administrative capital is called Gwangju, but it's not fully operational yet. Wait, I'm getting confused here.\n\nWait, the capital of South Korea is Seoul. The new administrative capital is called Gwangju, but it's still under construction. So the current capital is Seoul. I should clarify that. Also, sometimes people might confuse the capital with the administrative capital, but in this case, Seoul is the capital. Let me confirm. Yes, according to my knowledge, Seoul is the capital of South Korea

#### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성

input_variables를 사용하여, PromptTemplate를 생성한다.

**appendix: number of input_variables**: input_variables가 1개인 경우에는 invoke의 값이 자동으로 귿어가게 된다.  
하지만, 2개 이상인 경우에는 dictionary 형태로 전달해야 LLM에 Input으로 사용할 수 있다.

input_variables가 1개인 경우.

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

print(type(prompt))
print(prompt)

# chain 생성
chain = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content.split('</think>\n\n')[1]

<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['country'] input_types={} partial_variables={} template='{country}의 수도는 어디인가요?'


"<think>\nOkay, the user is asking for the capital of South Korea. I know that the capital is Seoul, but I should make sure I'm correct. Let me think... Yes, Seoul is the capital city of the Republic of Korea. It's also the largest city in the country. I should confirm that there's no confusion with other cities like Busan or Incheon. No, those are major cities but not the capital. So the answer is Seoul. I should present that clearly and maybe add a bit more info if needed, like its status as the political, cultural, and economic center. But the question is straightforward, so keeping it simple is better.\n</think>\n\n대한민국의 수도는 **서울**입니다.  \n서울은 한국의 정치, 문화, 경제 중심지로, 국가의 정부 기관과 주요 문화 시설이 집중되어 있습니다. 또한 한국의 가장 큰 도시이자 세계적으로 유명한 도시로 알려져 있습니다."

input_variables가 2개인 경우

In [ ]:
prompt = PromptTemplate(
    template="{country1}과 {country2}의 수도는 각각 어디인가요?",
    input_variables=["country1", "country2"],
)

# chain 생성
chain = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content.split('</think>\n\n')[1]

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT 

In [ ]:
bb

In [ ]:
chain.invoke({"country1": "대한민국", "country2": "일본"}).content.split('</think>\n\n')[1]

"<think>\nOkay, the user is asking about the capitals of South Korea and Japan. Let me start by recalling the information. South Korea's capital is Seoul, right? I remember that Seoul is a major city and the political, economic, and cultural center of the country. Now, for Japan, I think the capital is Tokyo. But wait, I should make sure I'm not mixing up any details. Sometimes people might confuse Tokyo with Kyoto, which is the former capital, but currently, Tokyo is the capital. Let me double-check that. Yes, Japan's capital is indeed Tokyo. So the answer should be Seoul for South Korea and Tokyo for Japan. I should present that clearly and maybe add a bit more context to confirm the accuracy.\n</think>\n\n대한민국(한국)의 수도는 **서울**이고, 일본의 수도는 **도쿄**입니다.  \n- **서울**은 한국의 정치, 경제, 문화 중심지로, 1948년부터 공식적인 수도로 지정되었습니다.  \n- **도쿄**는 일본의 정치, 경제, 문화 중심지로, 1868년 메이지 시대 이후 공식적인 수도로 지정되었습니다.  \n\n이 두 도시는 각국의 역사적, 문화적 중요성을 상징하는 중심지입니다."

partital_variables를 사용하여, dictionary형태로 input value와 default value를 적용할 수 있다.

In [ ]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

print(type(prompt))
print(prompt)

# chain 생성
chain = prompt | model

# country1 변수에 입력된 값이 자동으로 치환되어 수행됨 (input_variables가 1개 이므로.)
chain.invoke("대한민국").content.split('</think>\n\n')[1]

<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['country1'] input_types={} partial_variables={'country2': '미국'} template='{country1}과 {country2}의 수도는 각각 어디인가요?'


"<think>\nOkay, the user is asking for the capital cities of South Korea and the United States. Let me start by recalling the capitals. South Korea's capital is Seoul, right? I think that's correct. And the United States' capital is Washington, D.C. Wait, I should make sure I'm not mixing up any details. Sometimes people confuse Washington, D.C. with other cities like New York or Washington State. But no, the capital of the U.S. is definitely Washington, D.C. Let me double-check that. Yeah, Washington, D.C. is the capital, even though it's not a state. South Korea's capital is Seoul, which is also the largest city in the country. I don't think there's any confusion there. So the answer should be Seoul for South Korea and Washington, D.C. for the U.S. I should present that clearly.\n</think>\n\n대한민국의 수도는 **서울**이며, 미국의 수도는 **워싱턴 D.C.**(Washington, D.C.)입니다.  \n- **서울**은 대한민국의 정치, 경제, 문화 중심지로, 국가의 주요 기관이 집중되어 있습니다.  \n- **워싱턴 D.C.**는 미국의 정부 기관(예: 미국 정부청사, 국회, 대통령 주거지)이 위치한 도시로, 미국의 정치 중심지

**partitial variables**: partital_varirables에는 함수(Callable)의 값을 넣을 수 있지만, input_variables에는 함수는 안됩니다. 반드시 입력값으로 직접 값을 전달하여야 합니다.

partital variables에 함수값을 받는 대표적인 예시는 날짜를 반환하는 함수로서 사용법은 아래와 같습니다.

In [ ]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

# prompt 정의
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # 함수의 return값으로서 partial_variables값 전달.
    },
)

# 결과 확인
chain = prompt | model
chain.invoke(3).content.split('</think>\n\n')[1]

'<think>\nOkay, the user is asking for three famous people who have their birthday on June 28th, along with their birth dates. Let me start by recalling some notable individuals born on that date.\n\nFirst, I remember that Madonna was born on June 28, 1958. She\'s a major pop icon, so that\'s a good one. Next, maybe someone from the music industry. I think Elton John was born on March 25, but wait, that\'s not June 28. Maybe another artist? Oh, right, there\'s a singer named John Legend, but I think he was born in 1985. Wait, no, his birthday is March 1. Hmm. Maybe I need to check another area.\n\nWait, there\'s a famous person in the tech world. Steve Jobs was born on February 24, 1955, so that\'s not right. How about someone in sports? Maybe a former athlete? I\'m not sure. Wait, there\'s a famous actor named Matthew McConaughey, but his birthday is May 22. Not helpful.\n\nWait, maybe I should think of other fields. Oh, there\'s a famous musician named Bono from U2, but he was born o

함수값으로서 선언하여도, 아래와 같이 String으로 다시 값을 치환하여서 넣을 수 있다.

In [ ]:
chain.invoke({"today": "Jan 02", "n": 3}).content.split('</think>\n\n')[1]

'<think>\nOkay, the user is asking for three famous people who have their birthday on January 2nd, along with their birth dates. Let me start by recalling some well-known individuals born on that day.\n\nFirst, I remember that Bob Dylan was born on May 24, 1941, so that\'s not it. Wait, maybe someone else. Oh, right! Michael Jordan was born on February 17, 1963. Not January 2nd. Hmm.\n\nWait, there\'s a singer named Adele. Let me check her birth date. Adele is born on May 5, 1980. No. Maybe a different field. How about actors? I think there\'s an actor named James Gandolfini, but he was born on January 20, 1961. Close but not exactly.\n\nWait, maybe someone from the 19th century? Let me think. There\'s a famous person named John D. Rockefeller, but he was born on July 8, 1839. Not helpful.\n\nWait, I think there\'s a musician named Bob Seger. Let me check. Bob Seger was born on January 4, 1948. Still not. Maybe a more recent person. Oh! There\'s a singer named Justin Bieber. His birthd

#### 방법 3. File로서 값을 가져오는 방법.

LangChain에서 지원하는 PromptTemplate로서 file을 읽고 만드는 법은, 아래와 같이 load_prompt를 사용하는 방법이다.

In [ ]:
prompt = load_prompt('./data/fruit_color.yaml')
chain = prompt | model
chain.invoke("사과").content.split('</think>\n\n')[1]

'<think>\nOkay, the user is asking, "What color is a 사과?" which is Korean for "What color is an apple?" I need to answer that. First, I should confirm that "사과" is indeed the word for apple in Korean. Yes, that\'s correct.\n\nNow, the question is about the color of an apple. But wait, apples can vary in color depending on the variety. Common colors are red, green, yellow, or even pink. So the answer isn\'t a single color. The user might be expecting a specific answer, but the reality is that apples come in different colors. \n\nI should explain that while some apples are red, others are green, yellow, or even a mix of colors. Maybe mention that the color can depend on the type of apple. Also, note that the color can change as the apple ripens. For example, green apples might turn red when ripe. \n\nI should make sure to cover the main colors and maybe give examples of different apple varieties. Also, perhaps mention that the color can vary based on the region or the specific type. But 

### ChatPromptTemplate

ChatPromptTemplate는 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

```python
ChatPromptTemplate.from_messages(
    messages: List[Union[Tuple[str, str], BaseMessagePromptTemplate]],
    input_variables: Optional[List[str]] = None,
    partial_variables: Optional[Dict[str, Any]] = None
)
```

- <code>messages</code>: 메시지들을 순서대로 나열한 리스트. 각 메시지는 ("role", "template string") 형태의 튜플이거나 SystemMessagePromptTemplate, HumanMessagePromptTemplate 등의 객체로 구성할 수 있음.
- <code>input_variables</code>: {}로 들어가는 변수명들의 리스트 (자동 추론되므로 대부분 생략 가능)
- <code>partial_variables</code>: 일부 변수를 미리 값 또는 함수로 지정 (예: 날짜, 사용자 ID 등 고정값 처리용)

**즉 ChatPromptTemplate란, 이전 대화들의 주체를 Role로서 구성하고 나눈 대화들을 content로서 넣어서 // 자연스러운 대화 결과를 얻기 위한 Prompt라고 생각할 수 있다.**


#### PromptTemplate vs. ChatPromptTemplate

| 항목          | `PromptTemplate`                              | `ChatPromptTemplate`                                     |
|---------------|-----------------------------------------------|-----------------------------------------------------------|
| **형식**       | 일반 텍스트 (문장 하나)                        | Chat 형태 (역할 기반 메시지 나열)                          |
| **역할 설정**  | ❌ 없음                                       | ✅ `system`, `user`, `assistant` 등의 역할 지정 가능         |
| **사용 모델**  | 일반 LLM (`text-davinci`, `llama`, 등)        | Chat LLM (`gpt-3.5-turbo`, `gpt-4`, `claude` 등)           |
| **메시지 순서**| 하나의 문자열                                 | 여러 메시지를 순서대로 입력 가능                            |
| **구성 요소**  | `{}`로 변수를 치환하는 하나의 문자열            | `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate` 등 |



In [16]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

chain = chat_template | model
chain.invoke({'name':'Teddy', 'user_input': '당신의 이름은 무엇입니까?'}).content.split('</think>\n\n')[1]

'안녕하세요! 저는 테디입니다. 무엇을 도와드릴까요? 😊'

### MessagePlaceHolder

```python
MessagesPlaceholder(
    variable_name: str
)
```

MessagePlaceHolder는 특정 값을 넣을 때 사용된다. 보통 **이전 대화 기록**이나 **메모리 출력** 등을 넣는 데 사용된다. input_variables와 차이가 없어보이나 아래와 같은 차이가 있다.

#### input_variables vs. MessagePlaceHolder
| 항목 | 설명 |
|------|------|
| `input_variables` | `{}` 안에 들어갈 일반 문자열 값 (예: 숫자, 이름 등) |
| `MessagesPlaceholder` | 여러 개의 메시지 객체를 프롬프트 중간에 삽입할 때 사용 |

- `input_variables`는 문자열 템플릿을 채우는 용도
- `MessagesPlaceholder`는 Chat 구조 내에서 역할을 갖는 메시지들을 중간에 삽입할 수 있도록 함
- 보통 **이전 대화 기록**이나 **메모리 출력** 등을 넣는 데 사용됨


실제 MessagePlaceholder가 치환될 수 있는 값들은 아래와 같다. Input으로 들어가게 될때는 (role, message) 형태로 들어가게 된다.

| 클래스           | 역할 설명         | role 값       | 예시 |
|------------------|------------------|---------------|------|
| `HumanMessage`   | 사용자 입력 역할   | `"user"`       | `HumanMessage(content="안녕")` |
| `AIMessage`      | AI 응답 역할      | `"assistant"`  | `AIMessage(content="안녕하세요!")` |
| `SystemMessage`  | 시스템 지침 역할   | `"system"`     | `SystemMessage(content="너는 요약 AI야")` |

LLM의 Ouput을 다음 LLM의 Input으로 사용하고 싶을 때, PlaceHolder를 사용하는 법은 아래와 같다.

In [ ]:
# Step 1: 첫 번째 프롬프트 (단일 질문)
first_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 AI야."),
    ("human", "{question}"),
])

# 사용자 질문
user_question = "오늘 날씨 어때?"

# LLM에 첫 번째 질문 수행
first_result = model.invoke(first_prompt.format_messages(question=user_question))
print(first_result.content)

<think>
사용자의 질문에 대해 친절하게 대답해야 해. 오늘 날씨가 어때는 질문이기 때문에, 사용자에게 날씨 정보를 제공해야 해. 하지만 현실적으로 날씨 정보는 내가 직접 알 수 없어서, 사용자에게 도움을 드리기 위해 필요한 정보를 물어보는 것이 좋을까? 예를 들어, 사용자가 어떤 지역을 묻고 있는지, 또는 어떤 날씨 정보를 원하는지 확인하는 것이 좋을 것 같아. 하지만 사용자가 직접 날씨 정보를 원하는 것 같아서, 그럴 때는 사용자에게 해당 지역의 날씨를 확인해달라고 요청하는 것이 좋을 것 같아. 하지만 이 경우, 내가 직접 날씨 정보를 제공하지 못해서, 사용자에게 도움을 드리기 위해 필요한 정보를 묻는 것이 더 적절할 것 같아.

또한, 사용자와의 대화를 친절하게 유지하면서도, 실제 정보를 제공하지 않아도 되는 상황이기 때문에, 사용자에게 도움을 드리기 위해 필요한 정보를 묻는 것이 더 적절할 것 같아. 예를 들어, 사용자가 어떤 지역을 묻고 있는지, 또는 어떤 날씨 정보를 원하는지 확인하는 것이 좋을 것 같아. 이는 사용자가 더 정확한 정보를 얻도록 도와줄 수 있을 것 같아.

또한, 사용자가 직접 날씨 정보를 원하는 것 같아서, 그럴 때는 사용자에게 해당 지역의 날씨를 확인해달라고 요청하는 것이 좋을 것 같아. 하지만 이 경우, 내가 직접 날씨 정보를 제공하지 못해서, 사용자에게 도움을 드리기 위해 필요한 정보를 묻는 것이 더 적절할 것 같아.

결론적으로, 사용자가 날씨 정보를 원하는지 확인하고, 필요한 정보를 묻는 것이 적절할 것 같아. 예를 들어, "어떤 지역의 날씨를 알고 싶으신가요?"라고 물어보는 것이 좋을 것 같아.
</think>

안녕하세요! 오늘의 날씨 정보는 직접 확인해드릴 수 없어요. 혹시 특정 지역의 날씨를 알고 싶으신가요? 또는 어떤 날씨 정보(예: 기온, 강风, 비 등)를 원하시나요? 알려주시면 도와드릴게요! ☀️🌧️


In [ ]:
# Step 2: 생성된 응답을 기반으로 대화 기록 구성
first_chat_result = first_result.content.split('</think>\n\n')[1]

chat_history = [
    HumanMessage(content=user_question),
    AIMessage(content=first_chat_result)
]

# Step 3: 두 번째 프롬프트 - 이전 대화를 문맥으로 사용
second_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 요약 전문 AI야."),
    MessagesPlaceholder(variable_name="history"),  # 이전 메시지 들어갈 자리
    ("human", "방금 대화를 한 문장으로 요약해줘."),
])

# history에 이전 대화 기록 전달
second_result = model.invoke(
    second_prompt.format_messages(
        history=chat_history
    )
)

print(second_result.content.split('</think>\n\n')[1])

우리의 대화는 오늘의 날씨에 대한 질문과 함께 특정 지역이나 날씨 정보를 원하는지 확인하는 것이主要内容였습니다.


### FewShotPromptTemplate

```python
FewShotPromptTemplate(
    examples: List[dict],
    example_prompt: PromptTemplate,
    prefix: Optional[str] = None,
    suffix: Optional[str] = None,
    input_variables: List[str],
    example_separator: str = "\n\n"
)
```

| 인자명                 | 타입               | 설명                       |
| ------------------- | ---------------- | ------------------------ |
| `examples`          | `List[dict]`     | 예시 샘플 데이터 (입출력 쌍)        |
| `example_prompt`    | `PromptTemplate` | 각 예시를 어떻게 표현할지 정의        |
| `prefix`            | `str`            | 예시 앞에 붙는 도입 문구           |
| `suffix`            | `str`            | 실제 사용자 입력이 들어가는 부분       |
| `input_variables`   | `List[str]`      | `suffix`에서 사용되는 변수 목록    |
| `example_separator` | `str`            | 예시 사이의 구분자 (기본값: `\n\n`) |


FewShot이란 LLM에 실제 예시를 주어짐으로서, Output의 형태를 지정하거나 혹은 어떠한 형태를 원하는지 정보를 전달하여 할루시네이션을 줄일 수 있습니다. 실제 사용하는 예시는 아래와 같습니다.

In [ ]:
# Fewshow에 사용할 Examples를 선언합니다. 해당 값은 List[dict]입니다.
examples = [
    {"english": "apple", "korean": "사과"},
    {"english": "car", "korean": "자동차"},
    {"english": "school", "korean": "학교"}
]

# 각 예시를 LLM에 넣을 Prompt Template입니다.
example_prompt = PromptTemplate(
    input_variables=["english", "korean"], # 변수로 받을 값 입니다. english, korean을 입력으로 받습니다.
    template="Q: {english}\nA: {korean}" # 입력받은 값으로서 어떻게 출력할지 형태를 정합니다.
)

print('Single Dataset Check')
print(example_prompt.format(**examples[0]))

# FewShotTemplate 선언
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음은 영어 단어를 한국어로 번역한 예시입니다:\n",
    suffix="Q: {english}\nA:",
    input_variables=["english"],
    example_separator="\n\n"
)

# 실제 입력 넣기
print("\n\nFewShot Prompt 결과 확인.")
formatted_prompt = few_shot_prompt.format(english="computer")
print(formatted_prompt)

Single Dataset Check
Q: apple
A: 사과


FewShot Prompt 결과 확인.
다음은 영어 단어를 한국어로 번역한 예시입니다:


Q: apple
A: 사과

Q: car
A: 자동차

Q: school
A: 학교

Q: computer
A:


In [ ]:
# LLM에 사용할 Template 만들기
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영어 단어를 한국어로 번역해주는 AI입니다."),
    ("human", formatted_prompt)
])

print(model.invoke(chat_prompt.format_messages()).content.split('</think>\n\n')[1])

Q: computer  
A: 컴퓨터


### Example Selector

- 참조 사이트: https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/

Fewshot의 예시를 많이 넣어두고, RAG에서 선택해서 필요한 Examples를 가져다 쓰게 하는게 보통 방법이다.  
다양한 방법이 있지만, 몇몇 방법만 예제로서 확인해보자.


#### Step1. Dataset 생성.

실제 Example Selector에서 사용할 Dataset을 생성한다.

In [17]:
examples = [
    {"english": "apple", "korean": "사과"},
    {"english": "car", "korean": "자동차"},
    {"english": "school", "korean": "학교"},
    {"english": "computer", "korean": "컴퓨터"},
    {"english": "house", "korean": "집"},
    {"english": "pencil", "korean": "연필"},
    {"english": "teacher", "korean": "선생님"},
    {"english": "student", "korean": "학생"},
    {"english": "city", "korean": "도시"},
    {"english": "book", "korean": "책"}
]

**Appendix: Text to Embedding**: 실제 Embedding Model을 사용하는 것이 가장 정확하나. Local환경이 3090 GPU 1개로는 qwen3-4b + embedding model을 같이 올릴 수 없다. 따라서, dummy로 embedding하는 function으로서 embedding 값을 사용하자.

In [18]:
'''
Dummy Embedding Function
1. text의 앞부분 5글자만 가져와서 float으로 변경
2. embed_documents, embed_query는 Embeddings에서 정의된 함수로서 내부적으로 싸져서 자동으로 사용할 수 있게 한다.
'''

class DummyEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # 실제로는 Qwen3 임베딩 사용 필요
        return [[float(ord(c)) for c in text][:5] for text in texts]

    def embed_query(self, text: str) -> List[float]:
        return [float(ord(c)) for c in text][:5]
    
embedding = DummyEmbeddings()
embedding.embed_documents(texts=['english'])

[[101.0, 110.0, 103.0, 108.0, 105.0]]

**추가 팁: 어떤값이 Embedding 되는 것 인가?**

DummyEmbedding을 보게 되면, document를 embedding하고, query를 embedding하는 것을 알 수 있다. 어떤 값을 기준으로 Embedding하는지 알아보면 다음과 같다.

1. input_keys가 지정되었으면 그 key만 추출
2. 지정되지 않았으면 dict.keys()[0] (첫 번째 key) 사용
3. 해당 문자열들을 .embed_documents()로 전달
4. 나중에 .select_examples(input) 호출 시, embed_query(input)과 벡터 유사도를 비교하여 가장 가까운 예시 k개 반환

#### Step2. Example Selector 종류

실제로 많이 사용되는 ExampleSelector를 살펴보자.

| Selector 클래스                          | 설명                                     |
| ------------------------------------- | -------------------------------------- |
| `SemanticSimilarityExampleSelector`   | 입력값과 임베딩 유사도가 높은 예시 `k`개 선택 (가장 많이 사용) |
| `LengthBasedExampleSelector`          | 예시 길이 기반으로 `max_length` 이내에서 가능한 만큼 선택 |
| `MaxMarginalRelevanceExampleSelector` | 유사성과 다양성을 동시에 고려하여 예시 선택               |


#### Step3. SemanticSimilarityExampleSelector

실제, ExampleSelector중 하나로서, 해당 결과를 확인해보자.

```python
SemanticSimilarityExampleSelector.from_examples(
    examples: List[dict],
    embedding: Embeddings,
    vectorstore_cls: Type[VectorStore],
    k: int = 4,
    input_keys: Optional[List[str]] = None
)
```
| 인자명               | 타입                | 설명                                               |
| ----------------- | ----------------- | ------------------------------------------------ |
| `examples`        | `List[dict]`      | 예시 입력/출력 쌍들 (예: {"english": ..., "korean": ...}) |
| `embedding`       | `Embeddings` 객체   | 예시의 입력을 벡터로 변환할 임베딩 모델                           |
| `vectorstore_cls` | `VectorStore` 클래스 | 벡터를 저장하고 유사도 검색할 벡터스토어 클래스 (예: `FAISS`)          |
| `k`               | `int`             | 입력값과 유사한 예시 몇 개를 선택할지 결정                         |
| `input_keys`      | `List[str]` (선택)  | 어떤 key의 값을 임베딩 대상으로 사용할지 지정 (기본은 첫 키)            |



In [23]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

# Prompt Template으로서 english, korean을 Few Shot으로 넣기 위해 지정.
example_prompt = PromptTemplate(
    input_variables=["english", "korean"], # key값의 2개는 english, korean이다.
    template="Q: {english}\nA: {korean}" # 해당 값을 활용한 template는 다음과 같다.
)

# SemanticSearch로서 Embedding Vector를 구성한다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, # input dataset
    embeddings=DummyEmbeddings(),  # 현재 GPU Memory 부족으로서 Dummy Embedding Function을 사용하여 Embedding 한다.
    vectorstore_cls=FAISS, # 저장하는 vector store는 FAISS로서 지정한다.
    k=2 # Few Shot의 갯수를 2개로 지정한다.
)

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="다음은 영어 단어를 한국어로 번역한 예시입니다:\n",
    suffix="Q: {english}\nA:",
    input_variables=["english"]
)

print(few_shot_prompt.format(english="keyboard"))

다음은 영어 단어를 한국어로 번역한 예시입니다:


Q: pencil
A: 연필

Q: student
A: 학생

Q: keyboard
A:


In [25]:
# LLM에 사용할 Template 만들기
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영어 단어를 한국어로 번역해주는 AI입니다."),
    ("human", few_shot_prompt.format(english="keyboard"))
])

print(model.invoke(chat_prompt.format_messages()).content.split('</think>\n\n')[1])

Q: keyboard  
A: 키보드


**Appendix. multi_index for RAG**: 아래와 같이 여러 key값을 모두 embedding에 사용할 수 있다.

현재, DummyEmbedding에서는 2개 이상의 key값으로 Embedding하는 function을 지정하지 않아서, 사용하지는 못한다.

In [ ]:
# SemanticSearch로서 Embedding Vector를 구성한다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, # input dataset
    embeddings=DummyEmbeddings(),  # 현재 GPU Memory 부족으로서 Dummy Embedding Function을 사용하여 Embedding 한다.
    vectorstore_cls=FAISS, # 저장하는 vector store는 FAISS로서 지정한다.
    k=2, # Few Shot의 갯수를 2개로 지정한다.
    input_keys=["english", "korean"]
)

### LangChain-Hub

LangChain-Hub에 들어가게 되면, 아래 Figure와 같이, 남들이 사용한 Prompt를 확인하여 사용할 수 있다.
- link: https://smith.langchain.com/hub

![png](./img/1.png)

Prompt는 LangCahin-Hub에 연결하여, push하거나 pull하여 사용할 수 있다.

<hr>
참조: <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a><br>
참조: <a href="https://github.com/wjddyd66/LLM/tree/main">원본 코드</a><br>

코드에 문제가 있거나 궁금한 점이 있으면 wjddyd66@naver.com으로  Mail을 남겨주세요.